In [ ]:
epochs=10

# Teil 6 - Föderiertes Lernen auf MNIST mit einem CNN

## Upgrade auf Federated Learning in 10 Zeilen von PyTorch + PySyft


### Kontext

Federated Learning ist eine sehr aufregende und aufstrebende Technik des maschinellen Lernens, die darauf abzielt, Systeme aufzubauen, die auf dezentralen Daten lernen. Die Idee ist, dass die Daten in den Händen des Herstellers (der auch als _worker_ bezeichnet wird) verbleiben, was zur Verbesserung der Privatsphäre und des Eigentums beiträgt, und das Modell zwischen den Arbeitnehmern geteilt wird. Eine sofortige Anwendung besteht beispielsweise darin, das nächste Wort auf Ihrem Mobiltelefon vorherzusagen, wenn Sie Text schreiben: Sie möchten nicht, dass die für das Training verwendeten Daten - d. H. Ihre Textnachrichten - an einen zentralen Server gesendet werden.

Der Aufstieg von Federated Learning ist daher eng mit der Verbreitung des Datenschutzbewusstseins verbunden, und die DSGVO in der EU, die den Datenschutz seit Mai 2018 durchsetzt, hat als Katalysator gewirkt. Um die Regulierung vorwegzunehmen, haben große Akteure wie Apple oder Google massiv in diese Technologie investiert, insbesondere um die Privatsphäre der mobilen Benutzer zu schützen, aber sie haben ihre Tools nicht zur Verfügung gestellt. Wir bei OpenMined sind der Meinung, dass jeder, der bereit ist, ein Projekt für maschinelles Lernen durchzuführen, in der Lage sein sollte, Tools zum Schutz der Privatsphäre mit sehr geringem Aufwand zu implementieren. Wir haben Tools zum Verschlüsseln von Daten in einer einzigen Zeile erstellt [wie in unserem Blogbeitrag erwähnt](https://blog.openmined.org/training-cnns-using-spdz/) und veröffentlichen jetzt unser Federated Learning-Framework, das das nutzt Die neue PyTorch 1.0-Version bietet eine intuitive Benutzeroberfläche zum Erstellen sicherer und skalierbarer Modelle.

In diesem Tutorial verwenden wir direkt [das kanonische Beispiel für das Trainieren eines CNN auf MNIST mit PyTorch](https://github.com/pytorch/examples/blob/master/mnist/main.py) und zeigen, wie einfach es ist ist die Implementierung von Federated Learning mit unserer [PySyft-Bibliothek](https://github.com/OpenMined/PySyft/). Wir werden jeden Teil des Beispiels durchgehen und den Code unterstreichen, der geändert wird.

Sie finden dieses Material auch in [unserem Blogpost](https://blog.openmined.org/upgrade-to-federated-learning-in-10-lines).

Autoren:
- Théo Ryffel - GitHub: [@LaRiffle](https://github.com/LaRiffle)


Übersetzer:
- Vineet Jain - Github: [@vineetjai](https://github.com/vineetjai)

** Ok, lass uns anfangen! **

### Importe und Modellspezifikationen

Zuerst machen wir die offiziellen Importe

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

Und als die für PySyft spezifischen. Insbesondere definieren wir Remote Worker `Alice` und `Bob`.

In [2]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

Wir definieren die Einstellung der Lernaufgabe

In [9]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

### Laden und Senden von Daten an Mitarbeiter
Wir laden zuerst die Daten und wandeln den Trainingsdatensatz mit der Methode ".federate" in einen Verbunddatensatz um, der auf die Mitarbeiter aufgeteilt ist. Dieser Verbunddatensatz wird jetzt an einen Verbunddatenlader übergeben. Der Testdatensatz bleibt unverändert.

In [10]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

### CNN-Spezifikation
Hier verwenden wir genau das gleiche CNN wie im offiziellen Beispiel.

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### Definieren Sie die Zug- und Testfunktionen
Für die Zugfunktion müssen Sie das Modell für jede Charge an den richtigen Ort senden, da die Datenstapel auf `Alice` und `Bob` verteilt sind. Anschließend führen Sie alle Vorgänge remote mit derselben Syntax aus wie bei lokalem PyTorch. Wenn Sie fertig sind, erhalten Sie das aktualisierte Modell zurück und den Verlust, nach Verbesserungen zu suchen.

In [12]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

Die Testfunktion ändert sich nicht!

In [13]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### Starten Sie das Training!

In [14]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")




Train Epoch: 1 [0/60032 (0%)]	Loss: 2.305134
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.156802
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.896587
Train Epoch: 1 [5760/60032 (10%)]	Loss: 1.440329
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.867023
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.654317
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.593099
Train Epoch: 1 [13440/60032 (22%)]	Loss: 0.455322
Train Epoch: 1 [15360/60032 (26%)]	Loss: 0.371323
Train Epoch: 1 [17280/60032 (29%)]	Loss: 0.304580
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.314122
Train Epoch: 1 [21120/60032 (35%)]	Loss: 0.369122
Train Epoch: 1 [23040/60032 (38%)]	Loss: 0.238122
Train Epoch: 1 [24960/60032 (42%)]	Loss: 0.187751
Train Epoch: 1 [26880/60032 (45%)]	Loss: 0.523146
Train Epoch: 1 [28800/60032 (48%)]	Loss: 0.224569
Train Epoch: 1 [30720/60032 (51%)]	Loss: 0.143539
Train Epoch: 1 [32640/60032 (54%)]	Loss: 0.267939
Train Epoch: 1 [34560/60032 (58%)]	Loss: 0.187936
Train Epoch: 1 [36480/60032 (61%)]	Loss: 0.303569
Train Epoch:

Et voilà! Hier haben Sie ein Modell für Remote-Daten mit Federated Learning trainiert!

## Eine letzte Sache
Ich weiß, es gibt eine Frage, die Sie unbedingt stellen möchten: **Wie lange dauert es, Federated Learning im Vergleich zu normalem PyTorch durchzuführen?**

Die Rechenzeit ist tatsächlich **weniger als doppelt so lang** wie für die normale PyTorch-Ausführung ! Genauer gesagt dauert es 1.9-mal länger, was im Vergleich zu den Funktionen, die wir hinzufügen konnten, sehr wenig ist.

## Fazit

Wie Sie sehen, haben wir 10 Codezeilen geändert, um das offizielle Pytorch-Beispiel auf MNIST auf eine echte Federated Learning-Einstellung zu aktualisieren!

Natürlich gibt es Dutzende von Verbesserungen, die wir uns vorstellen können. Wir möchten, dass die Berechnung parallel für die Mitarbeiter ausgeführt wird und eine Verbundmittelung durchgeführt wird, um das zentrale Modell nur alle n Stapel zu aktualisieren, um die Anzahl der Nachrichten zu verringern, die wir für die Kommunikation zwischen Arbeitnehmern verwenden usw. Dies sind Funktionen, die wir verwenden. Wir arbeiten daran, Federated Learning für eine Produktionsumgebung vorzubereiten, und wir werden darüber schreiben, sobald sie veröffentlicht werden!

Sie sollten jetzt in der Lage sein, Federated Learning selbst durchzuführen! Wenn Ihnen dies gefallen hat und Sie sich der Bewegung zur Wahrung der Privatsphäre, zum dezentralen Besitz von KI und der KI-Lieferkette (Daten) anschließen möchten, können Sie dies auf folgende Weise tun!

### Star PySyft auf GitHub

Der einfachste Weg, unserer Community zu helfen, besteht darin, die Repositories zu markieren! Dies hilft, das Bewusstsein für die coolen Tools zu schärfen, die wir bauen.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Wählen Sie unsere Tutorials auf GitHub!

Wir haben wirklich nette Tutorials gemacht, um ein besseres Verständnis dafür zu bekommen, wie Federated and Privacy-Preserving Learning aussehen sollte und wie wir die Bausteine ​​dafür bauen.

- [Überprüfen Sie die PySyft-Tutorials](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### Mach mit bei unserem Slack!

Der beste Weg, um über die neuesten Entwicklungen auf dem Laufenden zu bleiben, ist, sich unserer Community anzuschließen!

- [Join slack.openmined.org](http://slack.openmined.org)

### Treten Sie einem Code-Projekt bei!

Der beste Weg, um zu unserer Community beizutragen, besteht darin, Code-Mitwirkender zu werden! Wenn Sie "einmalige" Miniprojekte starten möchten, können Sie auf der Seite PySyft GitHub Issues nach Problemen suchen, die mit "Good First Issue" gekennzeichnet sind.

- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Spenden

Wenn Sie keine Zeit haben, zu unserer Codebasis beizutragen, aber dennoch Unterstützung leisten möchten, können Sie auch Unterstützer unseres Open Collective werden. Alle Spenden fließen in unser Webhosting und andere Community-Ausgaben wie Hackathons und Meetups!

- [Spenden Sie über die Open Collective Page von OpenMined](https://opencollective.com/openmined)